In [1]:
#python3 t2_s1_tile.py --tof ts1_16x320_inv --ntile 16 --tsize 80

In [2]:
#makes simple tile from train_size1 images

In [3]:
import os, gc, subprocess, time, sys, shutil, argparse

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [4]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [5]:
from panda_bvv_config import *

## 1. Inputs ##

In [6]:
Zoom = 2
input_folder = gleason_path
input_mask_folder = mask_size1
to_folder = 'gs_proc_inv'

In [7]:
# ap = argparse.ArgumentParser()
# ap.add_argument('--ntile', help="number of tiles", dest = 'ntile', default = 16,\
#                 type=int)
# ap.add_argument('--tsize', help="size of one square tile", dest = 'tsize', default = 80,\
#                 type=int)# default for B3 net

#                 #("--cnn", type=str, help="training cnn name", dest = 'train_cnn')
# args = vars(ap.parse_args())
# ntile = args["ntile"]
# tsize = args["tsize"]

In [8]:
tsize = 80
ntile = 16

In [9]:
kw_for_tile_func = {'sz': tsize, 'N':ntile}
new_proc_folder = os.path.join(base_path, to_folder)

In [10]:
# last_train_label = 9000
# last_valid_label = 10617
#kw = {'gauss_blur':20}

In [11]:
# wrapping inputs:
if not os.path.exists(new_proc_folder):
    print("[INFO] 'creating {}' directory".format(new_proc_folder))
    os.makedirs(new_proc_folder)


[INFO] 'creating /run/media/admin/kagg/panda/gs_proc_inv' directory


## 2. Standard functions ##

In [12]:
#wrapper for image processing function
def process_all_images(proc_func):
    
    def wrapper(zoom = Zoom,
                input_biopsy_folder = input_folder,
                output_data_path = new_proc_folder,
                #df_name = train_labels,
                tif_file = False,
                with_mask = False,
                input_mask_folder = input_mask_folder,
                **kw):
        print(input_biopsy_folder)
        print(new_proc_folder)
        mask = None
        with tqdm(total=len(os.listdir(input_folder))) as pbar:
            for  filename in os.listdir(input_biopsy_folder):
                try:
                    biopsy = skimage.io.imread(os.path.join(input_biopsy_folder, filename))
                    shape = biopsy.shape
#                     if shape[0]<shape[1]: 
#                         biopsy = np.rot90(biopsy)
#                         shape = biopsy.shape 
                except:
                        print(f'can not proceed with {filename}')   
                if with_mask:
                    try:
                        mask = skimage.io.imread(os.path.join(input_mask_folder, img_num + '_mask.png'))
                        shape = mask.shape
                        if shape[0]<shape[1]: 
                            mask = np.rot90(mask)
                            shape = mask.shape 
                    except:
                        print('Failed to process mask:', filename)
                try:
                    data_new = proc_func(biopsy, **kw)
                    cv2.imwrite(os.path.join(output_data_path, filename), data_new)
                except Exception as ee:
                    print('Processing mistake:\n', ee, '\n', filename)            
                try:
                    del biopsy, mask, data_new, mask_new
                except:
                    pass
                pbar.update(1)
        gc.collect()
        return
    
    return wrapper

## 3. Processing functions ##

In [13]:
#@process_all_images
def tile16_simple(img, **kw):        
    sz=kw.get('sz') if kw.get('sz') else 32    
    N = kw.get('N') if kw.get('N') else 16
    final_dim = kw.get('final_dim') if kw.get('final_dim') else N**0.5 
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    a, b, c, d = img.shape
    data_new = (img.reshape(int(final_dim), int(final_dim),\
                                                    b, c, d).swapaxes(1,2).reshape(b*int(final_dim),\
                                                    c*int(final_dim), d))
    data_new = cv2.bitwise_not(data_new)
    
    return data_new

In [14]:
@process_all_images
def tile16_pad(img, **kw):        
    
    threshold = kw['threshold']
    porog = kw['porog']
    sz=kw.get('sz') if kw.get('sz') else 32    
    N = kw.get('N') if kw.get('N') else 16
    final_dim = kw.get('final_dim') if kw.get('final_dim') else N**0.5 
    grey_biopsy = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, white_mask = cv2.threshold(grey_biopsy, threshold, 255, 0)
    black_mask = cv2.bitwise_not(white_mask)
    img[black_mask==0] = (255,255,255)
    
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    
    try:
        yy = [y for y in idxs if img.reshape(img.shape[0],-1).sum(-1)[y] > porog] #4777000
        ee = idxs[:N-len(yy)]
        eee = np.pad(ee, [0,len(yy)], 'wrap' )
        np.random.shuffle(eee)
        assert len(ee)!=0, print('Dlina 0')
        img = img[eee]
    except:
        print('Ne srabotalo')
        return None
    a, b, c, d = img.shape
    
    data_new = (img.reshape(int(final_dim), int(final_dim),\
                                                    b, c, d).swapaxes(1,2).reshape(b*int(final_dim),\
                                                    c*int(final_dim), d))
    data_new = cv2.bitwise_not(data_new)
    #skimage.io.imshow(data_new)
    return data_new

#### Sandbox ####

In [15]:
# data_file = os.path.join(gleason_path, 'gs3', '69187b17ca2488b677f9a79e44654277.png')
# biopsy = skimage.io.imread(data_file)     
# skimage.io.imshow(biopsy) 

In [16]:
#tile16_pad(biopsy, threshold = 241, sz = 80, N =16)

## 4. Process input files for training purpose ##

In [17]:
for gsfolder in os.listdir(gleason_path):
    gsout = os.path.join(new_proc_folder, gsfolder)
    if not os.path.exists(gsout):
        print("[INFO] 'creating {}' directory".format(gsout))
        os.makedirs(gsout)
    tile16_pad(input_biopsy_folder = os.path.join(gleason_path, gsfolder),
                 output_data_path = gsout, threshold = 241, porog = 4700000,
               **kw_for_tile_func)

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] 'creating /run/media/admin/kagg/panda/gs_proc_inv/gs0' directory
/run/media/admin/kagg/panda/gs/gs0
/run/media/admin/kagg/panda/gs_proc_inv


323it [02:00,  2.68it/s]                     
  0%|          | 0/4 [00:00<?, ?it/s]

[INFO] 'creating /run/media/admin/kagg/panda/gs_proc_inv/gs3' directory
/run/media/admin/kagg/panda/gs/gs3
/run/media/admin/kagg/panda/gs_proc_inv


130it [00:43,  4.09it/s]                     

...

118it [00:26,  5.08it/s]

Ne srabotalo
Processing mistake:
 OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
 
 6875cac943f84443b8e7c2a44ee809f2.png
Ne srabotalo
Processing mistake:
 OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
 
 9079ffff621fc4e738b65b66587bfa47.png


126it [00:27,  5.10it/s]

Ne srabotalo
Processing mistake:
 OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
 
 5efaa05e7bf93a2c91c7c2592dcb8b64.png


128it [00:27,  4.59it/s]


In [18]:
import os
module_name = 't2_s2_gs_tile'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)